In [10]:
import unittest

import torch as th
import numpy as np

from active_critic.model_src.whole_sequence_model import WholeSequenceModel
from active_critic.policy.active_critic_policy import ActiveCriticPolicy
from active_critic.utils.test_utils import (make_acps, make_obs_act_space,
                                            make_wsm_setup)
from active_critic.utils.gym_utils import (DummyExtractor, make_dummy_vec_env,
                                           new_epoch_pap,
                                           new_epoch_reach)

from active_critic.utils.gym_utils import make_policy_dict, new_epoch_reach, make_dummy_vec_env, sample_expert_transitions, parse_sampled_transitions

In [11]:
def setup_ac():
        seq_len = 50
        d_output = 2
        obs_dim = 3
        batch_size = 2
        wsm_actor_setup = make_wsm_setup(
            seq_len=seq_len, d_output=d_output)
        wsm_critic_setup = make_wsm_setup(
            seq_len=seq_len, d_output=1)
        acps = make_acps(
            seq_len=seq_len, extractor=DummyExtractor(), new_epoch=new_epoch_pap)
        obs_space, acts_space = make_obs_act_space(
            obs_dim=obs_dim, action_dim=d_output)
        actor = WholeSequenceModel(wsm_actor_setup)
        critic = WholeSequenceModel(wsm_critic_setup)
        ac = ActiveCriticPolicy(observation_space=obs_space, action_space=acts_space,
                                actor=actor, critic=critic, acps=acps)
        return ac, acps, d_output, obs_dim, batch_size
def setup_ac_reach():
        seq_len = 50
        env, gt_policy = make_dummy_vec_env('reach', seq_len=seq_len)
        d_output = env.action_space.shape[0]
        wsm_actor_setup = make_wsm_setup(
            seq_len=seq_len, d_output=d_output)
        wsm_critic_setup = make_wsm_setup(
            seq_len=seq_len, d_output=1)
        acps = make_acps(
            seq_len=seq_len, extractor=DummyExtractor(), new_epoch=new_epoch_reach)
        actor = WholeSequenceModel(wsm_actor_setup)
        critic = WholeSequenceModel(wsm_critic_setup)
        ac = ActiveCriticPolicy(observation_space=env.observation_space, action_space=env.action_space,
                                actor=actor, critic=critic, acps=acps)
        return ac, acps, env, gt_policy

In [18]:
th.manual_seed(0)
ac, acps, env, _ = setup_ac_reach()
obsv = env.reset()
all_taken_actions = []
all_observations = [obsv]
all_scores_after = []
all_scores_before = []
ac.reset()
epsiodes = 2

for i in range(epsiodes*ac.args_obj.epoch_len):
    action = ac.predict(obsv)
    all_taken_actions.append(action)
    obsv, rew, dones, info = env.step(action)
    all_observations.append(obsv)
    all_scores_after.append(ac.current_result.expected_succes_after)
    all_scores_before.append(ac.current_result.expected_succes_before)
    if (i+1) % 5 == 0:
        all_observations = [obsv]

all_scores_after_th = th.tensor(np.array([s.detach().cpu().numpy() for s in all_scores_after]).reshape(
    [epsiodes, ac.args_obj.epoch_len, ac.args_obj.epoch_len, 1]), device=ac.args_obj.device)
all_scores_before_th = th.tensor(np.array([s.detach().cpu().numpy() for s in all_scores_before]).reshape(
    [epsiodes, ac.args_obj.epoch_len, ac.args_obj.epoch_len, 1]), device=ac.args_obj.device)


/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [12]:
th.manual_seed(0)

current_step = 1
ac, acps, act_dim, obs_dim, batch_size =  setup_ac()

org_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
opt_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2 * th.ones([batch_size, current_step + 1, obs_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
org_obs_seq = 2 * th.ones([batch_size, current_step + 1, obs_dim],
                            device=acps.device, dtype=th.float, requires_grad=False)

actions, expected_success = ac.optimize_act_sequence(
    actions=opt_actions, observations=obs_seq, current_step=current_step, stop_opt=False, opt_end=False)
assert th.equal(
org_actions[:, :current_step], actions[:, :current_step]), 'org_actions were overwritten'

assert not th.equal(actions[:, current_step:], org_actions[:,
                                                                current_step:]),'seq optimisation did not change the actions'
assert th.all(
expected_success[:, -1] >= ac.args_obj.optimisation_threshold), 'optimisation does not work.'
assert th.equal(obs_seq, org_obs_seq), 'Observations were changed.'

In [16]:
ac.history.opt_scores

[]

In [4]:
th.manual_seed(1)
current_step = 1

ac, acps, act_dim, obs_dim, batch_size =  setup_ac()
opt_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2 * th.ones([batch_size, current_step + 1, obs_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
obs_seq[0] *= 2
critic_input = ac.get_critic_input(acts=opt_actions, obs_seq=obs_seq)
critic_scores = ac.critic.forward(critic_input)
actions, expected_success_nonstop = ac.optimize_act_sequence(
    actions=opt_actions, observations=obs_seq, current_step=current_step, stop_opt=False, opt_end=False)

th.manual_seed(1)
ac, acps, act_dim, obs_dim, batch_size =  setup_ac()
opt_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2 * th.ones([batch_size, current_step + 1, obs_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
obs_seq[0] *= 2
critic_input = ac.get_critic_input(acts=opt_actions, obs_seq=obs_seq)
critic_scores = ac.critic.forward(critic_input)
actions, expected_success = ac.optimize_act_sequence(
    actions=opt_actions, observations=obs_seq, current_step=current_step, stop_opt=True, opt_end=False)

assert (expected_success[:,-1] < expected_success_nonstop[:,-1]).sum() > 0
assert (expected_success[:,-1] < expected_success_nonstop[:,-1]).sum() < len(expected_success)

In [5]:
th.manual_seed(1)
current_step = 1

ac, acps, act_dim, obs_dim, batch_size =  setup_ac()
opt_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2 * th.ones([batch_size, current_step + 1, obs_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
obs_seq[0] *= 2
critic_input = ac.get_critic_input(acts=opt_actions, obs_seq=obs_seq)
critic_scores = ac.critic.forward(critic_input)
actions, expected_success_nonstop = ac.optimize_act_sequence(
    actions=opt_actions, observations=obs_seq, current_step=current_step, stop_opt=False, opt_end=False)

th.manual_seed(1)
ac, acps, act_dim, obs_dim, batch_size =  setup_ac()
opt_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2 * th.ones([batch_size, current_step + 1, obs_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
obs_seq[0] *= 2
critic_input = ac.get_critic_input(acts=opt_actions, obs_seq=obs_seq)
critic_scores = ac.critic.forward(critic_input)
actions, expected_success = ac.optimize_act_sequence(
    actions=opt_actions, observations=obs_seq, current_step=current_step, stop_opt=False, opt_end=True)

assert (max(expected_success[:,-1]) > max(expected_success_nonstop[:,-1]))

In [6]:
th.manual_seed(1)
current_step = 1

ac, acps, act_dim, obs_dim, batch_size =  setup_ac()
opt_actions = th.zeros([batch_size, acps.epoch_len, act_dim],
                        device=acps.device, dtype=th.float, requires_grad=True)
obs_seq = 2 * th.ones([batch_size, current_step + 1, obs_dim],
                        device=acps.device, dtype=th.float, requires_grad=False)
obs_seq[0] *= 2
critic_input = ac.get_critic_input(acts=opt_actions, obs_seq=obs_seq)
critic_scores = ac.critic.forward(critic_input)
actions, expected_success = ac.optimize_act_sequence(
    actions=opt_actions, observations=obs_seq, current_step=current_step, stop_opt=True, opt_end=True)